In [1]:
import pandas as pd
import seaborn as sns
import ast
from scipy.stats import spearmanr, zscore
from preprocessing import get_participant_names

# global variables
data_path = 'data/'
rank_result_path = 'rank-result/'

# get participant names
names = get_participant_names(data_path)
print('participants names: ', names)
print('participant num: ', len(names))

# generate rank
if not os.listdir(rank_result_path):
    from rank_utility import *
    # - click rank
    click_rank_files = [f for f in os.listdir(data_path) if "click_rank" in f]
    generate_click_rank(data_path, click_rank_files)
    # - general rank
    pairwise_rank_files = [f for f in os.listdir(data_path) if "pairwise_rank" in f]
    generate_pairwise_rank(data_path, pairwise_rank_files)

participants names:  ['mh', 'yz', 'an', 'co', 'jt', 'xiao', 'hw', 'ts', 'hu', 'huy', 'yx', 'lf', 'dao', 'mr', 'yq', 'lr']
participant num:  16


# Single Analysis

In [2]:
click_rank_data = pd.read_csv(rank_result_path + 'click_rank_result.csv')
pairwise_rank_data = pd.read_csv(rank_result_path + 'pairwise_rank_result.csv')
print(click_rank_data.dtypes, '\n')
print(pairwise_rank_data.dtypes)

tester               object
click_grid_1_rank    object
click_grid_2_rank    object
dtype: object 

tester             object
pwc_grid_1_rank    object
pwc_grid_1_2       object
pwc_grid_2_rank    object
pwc_grid_1_w       object
dtype: object


In [3]:
name = 'huy'

row = click_rank_data[click_rank_data['tester'] == name]
clk_1 = ast.literal_eval(row['click_grid_1_rank'].values[0])
clk_2 = ast.literal_eval(row['click_grid_2_rank'].values[0])

row = pairwise_rank_data[pairwise_rank_data['tester'] == name]
pw_1 = ast.literal_eval(row['pwc_grid_1_rank'].values[0])[0:-2]
pw_2 = ast.literal_eval(row['pwc_grid_2_rank'].values[0])[0:-2]

cor_1, pval_1 = spearmanr(clk_1, pw_1)
print(spearmanr(clk_1, pw_1))
print(spearmanr(pw_1, clk_1))
print(cor_1, pval_1)

SpearmanrResult(correlation=0.4285714285714286, pvalue=0.3965014577259474)
SpearmanrResult(correlation=0.4285714285714286, pvalue=0.3965014577259474)
0.4285714285714286 0.3965014577259474


In [4]:
dist_rank_data = pd.read_csv(rank_result_path + 'dist_rank.csv')
row = dist_rank_data[dist_rank_data['name'] == name]
dist_1 = ast.literal_eval(row[row['event'] == 'gaze_grid_1']['rank'].values[0])
dist_2 = ast.literal_eval(row[row['event'] == 'gaze_grid_2']['rank'].values[0])

# Batch Analysis

In [5]:
result = []

click_rank_data = pd.read_csv(rank_result_path + 'click_rank_result.csv')
pairwise_rank_data = pd.read_csv(rank_result_path + 'pairwise_rank_result.csv')
dist_rank_data = pd.read_csv(rank_result_path + 'dist_rank.csv')

## Correlation between Click Rank and Pairwise Rank

In [6]:
for name in names:
    row = click_rank_data[click_rank_data['tester'] == name]
    clk_1 = ast.literal_eval(row['click_grid_1_rank'].values[0])
    clk_2 = ast.literal_eval(row['click_grid_2_rank'].values[0])

    row = pairwise_rank_data[pairwise_rank_data['tester'] == name]
    pw_1 = ast.literal_eval(row['pwc_grid_1_rank'].values[0])[0:-2]
    pw_2 = ast.literal_eval(row['pwc_grid_2_rank'].values[0])[0:-2]

    row = dist_rank_data[dist_rank_data['name'] == name]
    dist_1 = ast.literal_eval(row[row['event'] == 'gaze_grid_1']['rank'].values[0])
    dist_2 = ast.literal_eval(row[row['event'] == 'gaze_grid_2']['rank'].values[0])
    
    cor_1, pval_1 = spearmanr(clk_1, pw_1)
    cor_2, pval_2 = spearmanr(clk_2, pw_2)
    
    result.append({'name': name,
                   'grid_1_cor': cor_1,
                   'grid_1_pval': pval_1,
                   'grid_2_cor': cor_2,
                   'grid_2_pval': pval_2,})

result = pd.DataFrame(result, columns=['name', 'grid_1_cor', 'grid_1_pval', 'grid_2_cor', 'grid_2_pval'])

c1 = result['grid_1_cor']
c2 = result['grid_2_cor']
cor = c1.append(c2, ignore_index=True)
sns.distplot(cor, bins=15, kde=False)

<AxesSubplot:>

In [7]:
cor.describe()

count    32.000000
mean      0.578571
std       0.409177
min      -0.200000
25%       0.257143
50%       0.685714
75%       0.942857
max       1.000000
dtype: float64

In [8]:
if True not in (abs(zscore(cor)) >= 3): print('no outlier')
else: print('outlier detected')

no outlier


## Correlation between Fixation Distribution Rank and Pairwise Rank

In [4]:
for name in names:
    row = pairwise_rank_data[pairwise_rank_data['tester'] == name]
    pw_1 = ast.literal_eval(row['pwc_grid_1_rank'].values[0])[0:-2]
    pw_2 = ast.literal_eval(row['pwc_grid_2_rank'].values[0])[0:-2]

    row = dist_rank_data[dist_rank_data['name'] == name]
    dist_1 = ast.literal_eval(row[row['event'] == 'gaze_grid_1']['rank'].values[0])
    dist_2 = ast.literal_eval(row[row['event'] == 'gaze_grid_2']['rank'].values[0])
    
    cor_1, pval_1 = spearmanr(dist_1, pw_1)
    cor_2, pval_2 = spearmanr(dist_2, pw_2)
    
    result.append({'name': name,
                   'grid_1_cor': cor_1,
                   'grid_1_pval': pval_1,
                   'grid_2_cor': cor_2,
                   'grid_2_pval': pval_2,})

result = pd.DataFrame(result, columns=['name', 'grid_1_cor', 'grid_1_pval', 'grid_2_cor', 'grid_2_pval'])

c1 = result['grid_1_cor']
c2 = result['grid_2_cor']
cor = c1.append(c2, ignore_index=True)
sns.distplot(cor, bins=15, kde=False)

TypeError: '<' not supported between instances of 'dict' and 'dict'